                                        SLUČAJNE ŠUME
                                        
   Postoje različiti pristupi  mašinskom učenju, a jedan od njih uključuje veći broj modela koji zajednički donose odluke. Osnovna ideja koja se krije iza ovog pristupa je da su 'dve glave pametnije od jedne', odnosno
više modela omogućavaju bolju precinost nego jedan.
   

Metod slučajnih šuma je zasnovan na prostoj agregaciji stabala odlučivanja. 
Svaki čvor stabla sadrži po jedan test koji očekuje više od jednog ishoda.
A svakom ishodu  odgovara jedna grana stabla koja vodi ka sledećem čvoru. 
U listovima se ne nalaze testovi već u njima se nalaze predviđanja.
Jasno je da šumu čine više ovakvih stabala testiranih na različitim podskupovima skupa za obučavanje. 
Bitno je da se svako stablo obučava na različitom podskupu kako bi greške bili slabije korelisane.
   

Za početak razmotrićemo šta nam biblioteka scikit-learn nudi...

In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn import model_selection
from sklearn import preprocessing
from sklearn import metrics

from sklearn import ensemble

In [21]:
data = pd.read_csv('diabetes.csv')
y = data['Outcome']
X = data.drop(columns=['Outcome'], axis=1 )
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, stratify=y, random_state = 7)
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
model_forest = ensemble.RandomForestClassifier(n_estimators=20, max_depth=3, random_state=7)
model_forest.fit(X_train, y_train)
y_predicted = model_forest.predict(X_test)
metrics.accuracy_score(y_test, y_predicted)


0.7598425196850394

In [4]:
import csv
from math import sqrt
from random import randrange
from random import seed

In [5]:
# Create a random subsample from the dataset with replacement
#slučajni podskup
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

 Koristicemo Ginijev indeks kao meru homogenosti skupa instanci.
     Ovaj pristup pociva na udelima isntanci razlicitih klasa u ukupnom skupu.
 Ginijev indeks je definisan formulom: 
 G(p1, p2, ..., pk) = 1 - (p1^2 + p2^2 + ... + pk^2) = 1 - sum(pi^2)

In [6]:
def giniIndex(groups,classes):
    n_instances = float(sum([len(group) for group in groups]))
    Gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0.0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p*p
        Gini += (1.0-score) * (size / n_instances)
    return Gini

In [7]:
#delimo podatke
def testSplit(index,value,dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index]< value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [8]:
# pomocu Ginjevog indeksa biramo najbolji tacku za podelu skupa podataka
def getSplitForest(dataset, n_attributes):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    attributes = list()
    while len(attributes) < n_attributes:
        index = randrange(len(dataset[0])-1)
        if index not in attributes:
            attributes.append(index)
    for index in attributes:
        for row in dataset:
            groups = testSplit(index, row[index], dataset)
            Gini = giniIndex(groups, class_values)
            #print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], Gini))
            if Gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], Gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [9]:
# formiramo vrednost zavrsnog svora
def toTerminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [10]:
# funkcija za deljenje cvorova
def splitNodeForest(node, max_depth, min_size, n_attributes, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = toTerminal(left + right)
        return
    #pravimo zavrsni cvor ukoliko smo dosli do maksimalne dubine
    if depth >= max_depth:
        node['left'], node['right'] = toTerminal(left), toTerminal(right)
        return
    #proveravamo da li je postignuta minimalna veličina ili je potrebno dodatno račvanje stabla
    
    if len(left) <= min_size:
        node['left'] = toTerminal(left)
    else:
        node['left'] = getSplitForest(left, n_attributes)
        splitNodeForest(node['left'], max_depth, min_size, n_attributes,depth+1)
   
    if len(right) <= min_size:
        node['right'] = toTerminal(right)
    else:
        node['right'] = getSplitForest(right, n_attributes)
        splitNodeForest(node['right'], max_depth, min_size, n_attributes,  depth+1)

In [11]:
# formiramo stablo odluke
def buildTree(train, max_depth, n_attributes ,min_size):
    root = getSplitForest(train, n_attributes)
    splitNodeForest(root, max_depth, min_size, n_attributes ,1)
    return root


In [12]:
# vršimo predikciju na osnovu stabla odluke
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [13]:
# vršimo predikciju na osnovu liste predikcija
def baggingPredict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

In [14]:
# Random Forest Algorithm
def randomForest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for _ in range(n_trees):
        sample = subsample(train, sample_size)
        tree = buildTree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [baggingPredict(trees, row) for row in test]
    return(predictions)

In [15]:
##Učitavamo fajl, red po red
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [16]:
## konvertujemo nisku u float
def strToFloat(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
#konvertujemo nisku u intidžer
def strToInt(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [52]:
########### Normalizacija podataka ###########
## TODO: dodati teks o kros validaciji
##pronalazimo najmanju i najveću vrednost u svakoj koloni
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        colvalues = [row[i] for row in dataset]
        min_value = min(colvalues) 
        max_value = max(colvalues)
        minmax.append([min_value, max_value])
    return minmax

#Normalizujemo skup podataka
def Normalize_Dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [17]:
########### Kros validacija ###########
## TODO: dodati tekst o kros validaciji

#delimo skup na k polja - kros validacija
def crossValidationSplit(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


# ocenjivanje algoritma koristeci kros-validaciju na dati skup podataka i 
# i naš randomForest algoritam uz pomoć funkcije za precenu učinka:getAccuracy
def evaluateAlgorithm(dataset, algorithm, n_folds, performance_assessment, *args):
    folds = crossValidationSplit(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        performance = performance_assessment(actual, predicted)
        scores.append(performance)
    return scores

In [18]:
# dobijena tacnost predvidjanja #
def getAccuracy(actual,predicted):
    correct = 0
    for x in range(len(actual)):
        if actual[x] == predicted[x]:
            correct += 1
    return correct / float(len(actual)) * 100.00

In [ ]:
def main():
    seed(2)
    filename =  'diabetes.csv'
    dataset = load_csv(filename)
    dataset = dataset[1:] 
    for i in range(0, len(dataset[0])):
        strToFloat(dataset, i)
    n_folds = 10
    max_depth = 3
    min_size = 3.0
    sample_size = 3.0
    n_features = 20 
    for n_trees in [1,2,4,8, 20]:
        scores = evaluateAlgorithm(dataset, randomForest, n_folds, getAccuracy,max_depth, min_size, sample_size, n_trees, n_features)
        print('Trees: %d' % n_trees)
        print('Scores: %s' % scores)
        print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

main()

Trees: 1
Scores: [65.78947368421053, 68.42105263157895, 75.0, 68.42105263157895, 77.63157894736842, 60.526315789473685, 75.0, 68.42105263157895, 76.31578947368422, 65.78947368421053]
Mean Accuracy: 70.132%
Trees: 2
Scores: [73.68421052631578, 63.1578947368421, 76.31578947368422, 69.73684210526315, 80.26315789473685, 72.36842105263158, 72.36842105263158, 72.36842105263158, 65.78947368421053, 75.0]
Mean Accuracy: 72.105%
